<a href="https://colab.research.google.com/github/thmswhelan/GAA_Rankings/blob/main/Gaa_Football_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import requests
from bs4 import BeautifulSoup
import csv

# URL for GAA fixtures or competition page
url = "https://www.gaa.ie/fixtures-results"

# Send HTTP request to fetch the page content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    page_content = response.content
    soup = BeautifulSoup(page_content, "html.parser")

    # Prepare CSV file to save the data
    with open('gaa_matches.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write headers to the CSV
        writer.writerow(["Competition", "Home Team", "Away Team", "Date", "Venue", "Home Score", "Away Score"])

        # Extract match details
        match_details = soup.find_all("div", class_="gar-match-item")

        for match in match_details:
            # Example of extracting team names
            home_team = match.find("div", class_="gar-match-item__team -home")
            away_team = match.find("div", class_="gar-match-item__team -away")

            # Extract the competition name for each match (assuming it may be inside the match item)
            competition = match.find_previous("h3", class_="gar-matches-list__group-name")
            competition_name = competition.get_text(strip=True) if competition else "Unknown Competition"

            # Extract the date of the match
            match_date = match.get("data-match-date", "No date available")

            # Extract the venue for the match
            venue = match.find("div", class_="gar-match-item__venue")
            venue_name = venue.get_text(strip=True) if venue else "No venue available"

            # Extract the home and away scores
            scores = match.find_all("div", class_="gar-match-item__score")

            # If scores are found, we process them correctly
            if len(scores) == 2:
                home_score = scores[0].get_text(strip=True)
                away_score = scores[1].get_text(strip=True)
            else:
                home_score = "N/A"
                away_score = "N/A"

            # If the score contains a date, we make sure to extract only the score
            if '-' not in home_score or '-' not in away_score:
                home_score = "Invalid score"
                away_score = "Invalid score"

            # Get team names
            if home_team and away_team:
                home_team_name = home_team.get_text(strip=True)
                away_team_name = away_team.get_text(strip=True)
            else:
                home_team_name = "Unknown"
                away_team_name = "Unknown"

            # Write each match's data to the CSV file
            writer.writerow([competition_name, home_team_name, away_team_name, match_date, venue_name, home_score, away_score])

    print("Data saved to 'gaa_matches.csv'.")
else:
    print("Failed to retrieve the page. Status Code:", response.status_code)



Data saved to 'gaa_matches.csv'.


In [44]:
!pkill -f chromium-browser || true

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

chrome_options = Options()
chrome_options.binary_location = "/usr/bin/chromium-browser"  # Important!
chrome_options.add_argument("--headless")  # Necessary in many cases
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--remote-debugging-port=9222")  # Important for Chromium
chrome_options.add_argument("--user-data-dir=/tmp/chrome-temp")  # Clean profile

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the GAA fixtures/results page
driver.get("https://www.gaa.ie/fixtures-results/")

# List of months and years you want
years = [2024, 2025]
months = list(range(1, 13))  # January (1) to December (12)

all_matches = []

for year in years:
    for month in months:
        # Select year
        year_select = Select(driver.find_element(By.NAME, 'year'))
        year_select.select_by_value(str(year))

        # Select month
        month_select = Select(driver.find_element(By.NAME, 'month'))
        month_select.select_by_value(str(month))

        # Wait for the page to reload after selecting
        time.sleep(2)  # you might need to adjust timing

        # Now scrape the page
        matches = driver.find_elements(By.CLASS_NAME, "gar-match-item")

        for match in matches:
            try:
                competition = match.find_element(By.CLASS_NAME, "gar-match-item__competition").text
                home_team = match.find_element(By.CLASS_NAME, "gar-match-item__team.-home").text
                away_team = match.find_element(By.CLASS_NAME, "gar-match-item__team.-away").text
                scores = match.find_elements(By.CLASS_NAME, "gar-match-item__score")
                home_score = scores[0].text if len(scores) > 0 else ''
                away_score = scores[1].text if len(scores) > 1 else ''
                date = match.find_element(By.CLASS_NAME, "gar-match-item__date").text
                venue = match.find_element(By.CLASS_NAME, "gar-match-item__venue").text

                all_matches.append({
                    'Competition': competition,
                    'Home Team': home_team,
                    'Home Score': home_score,
                    'Away Team': away_team,
                    'Away Score': away_score,
                    'Date': date,
                    'Venue': venue,
                    'Month': month,
                    'Year': year
                })
            except Exception as e:
                print(f"Error extracting match: {e}")

# Close the browser
driver.quit()

# Save all data to a CSV
df = pd.DataFrame(all_matches)
df.to_csv("gaa_fixtures_results.csv", index=False)


^C


WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (chrome not reachable)
  (The process started from chrome location /usr/bin/chromium-browser is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x579f6d0434e3 <unknown>
#1 0x579f6cd72c76 <unknown>
#2 0x579f6cd9bd78 <unknown>
#3 0x579f6cd98029 <unknown>
#4 0x579f6cdd6ccc <unknown>
#5 0x579f6cdd647f <unknown>
#6 0x579f6cdcdde3 <unknown>
#7 0x579f6cda32dd <unknown>
#8 0x579f6cda434e <unknown>
#9 0x579f6d0033e4 <unknown>
#10 0x579f6d0073d7 <unknown>
#11 0x579f6d011b20 <unknown>
#12 0x579f6d008023 <unknown>
#13 0x579f6cfd61aa <unknown>
#14 0x579f6d02c6b8 <unknown>
#15 0x579f6d02c847 <unknown>
#16 0x579f6d03c243 <unknown>
#17 0x790bb9273ac3 <unknown>
